In [1]:
%load_ext autoreload
import sys
sys.path.append('../../..')
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.interpolate import interp1d, interp2d

import config
import main_components as main
import pickle
import darkhistory.physics as phys
from darkhistory.spec.spectra import Spectra
from darkhistory.spec.spectrum import Spectrum
import darkhistory.low_energy.atomic as atomic
import darkhistory.spec.spectools as spectools
from plotting_utils import plot_distortion, download_plot

import matplotlib
matplotlib.rc_file('../../../matplotlibrc')
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.colors as colors
%matplotlib inline

In [2]:
### SETTINGS TO CHANGE
output_dir = '/Users/viviesque/OneDrive - Massachusetts Institute of Technology/DarkHistory/output/'
fig_dir = '/Users/viviesque/OneDrive - Massachusetts Institute of Technology/DarkHistory/figs/'
results_comp = pickle.load(open(output_dir+f'component_distortions.p','rb'))

# Generate data, if not loading previous

In [3]:
# Parameters
nmax = 10
start_rs, high_rs, end_rs = 3e3, 1.555e3, 4
cf = 16
rtol = 1e-6

# DM model
pri, DM_process = 'elec', 'decay'
mDM = 2e8
lifetime=2e25

In [ ]:
%autoreload
results_comp = main.evolve(
    DM_process=DM_process, mDM=mDM,
    lifetime=lifetime,
    primary=pri+'_delta',
    start_rs=start_rs, high_rs=high_rs, end_rs=end_rs,
    reion_switch=True, reion_method='Puchwein', heat_switch=True,
    coarsen_factor=cf,
    distort=True, recfast_TLA=True,
    nmax=nmax, rtol=rtol
)

pickle.dump(results_comp, open(output_dir+'component_distortions.p', 'wb'))

****** Loading transfer functions... ******
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******


../../../darkhistory/spec/spectrum.py:861: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)


# Get separate distortion components

In [ ]:
# Dictionary that collects all of the separate components of the distortion
components = {}

In [ ]:
# ICS component
ICS_component = results_comp['lowengphot'].copy()
ICS_component._grid_vals *= list(map(int, ICS_component.eng < phys.rydberg))
ICS_component.redshift(1)
components['ICS'] = {'distortion': ICS_component.sum_specs()}

In [ ]:
# Reionization and recombination components
weights_recomb = (results_comp['lowengphot'].rs > 400)
weights_reion = results_comp['lowengphot'].rs < 400

atomic_component = results_comp['dist_atomic'].copy()
atomic_component.redshift(1)
components['recomb'] = {'distortion': atomic_component.sum_specs(weight=weights_recomb)}
components['reion'] = {'distortion': atomic_component.sum_specs(weight=weights_reion)}

In [ ]:
# y-type from heating
yheat_component = results_comp['dist_yheat'].copy()
yheat_component.redshift(1)
components['y_heat'] = {'distortion': yheat_component.sum_specs()}

In [ ]:
### Check that sum of components is equal to total distortion
# total_spec = Spectrum(components['ICS']['distortion'].eng, 
#                       np.zeros_like(components['ICS']['distortion'].eng), 
#                       rs=1, spec_type='N')

# components['recomb']['distortion'].rebin(components['ICS']['distortion'].eng)
# components['reion']['distortion'].rebin(components['ICS']['distortion'].eng)
# components['y_heat']['distortion'].rebin(components['ICS']['distortion'].eng)

# total_spec.N += components['ICS']['distortion'].N
# total_spec.N += components['recomb']['distortion'].N
# total_spec.N += components['reion']['distortion'].N
# total_spec.N += components['y_heat']['distortion'].N

# components['total'] = {'distortion': total_spec}

# Plot

In [ ]:
cmap = cm.plasma(np.linspace(0.2,0.85,4))

plt.figure(figsize=(7,7))
ax = plt.gca()

plot_distortion(ax, components['recomb'], label=r'$z > 400$',
                color=cmap[0], leg=True, first=False)
plot_distortion(ax, components['reion'], label=r'$z < 400$',
                color=cmap[1], leg=True, first=False)
plot_distortion(ax, components['ICS'], label='ICS',
                color=cmap[2], leg=True, first=False)
plot_distortion(ax, components['y_heat'], label='heat',
                color=cmap[3], leg=True, first=False)
# plot_distortion(ax, components['total'], label='sum?',
#                 color='grey', leg=True, first=False)
plot_distortion(ax, results_comp, label='full', color='k', leg=True, first=False)